In [5]:
import pandas as pd
import re
import numpy as np

In [6]:
df = pd.read_csv("DF_BOOTCAMPS_2591.csv")

In [7]:
#CONSERVAR A TODOS AQUELLOS QUE NO TENGAN MISSING VALUES EN EDUCATION Y CERTIFICATION. 

dfe = df[df["education"].notna()] #DATAFRAME DE EDUCATION
dfe = dfe.reset_index(drop = True) #REINICIAR INDICE

dfc = df[df["certifications"].notna()] #DATAFRAME DE CERTIFICACIÓN

In [8]:
dfe.columns

Index(['id', 'scraping_services_id', 'linkedin_id', 'linkedin_profile_url',
       'last_name', 'first_name', 'v_extension', 'location', 'job_title',
       'company', 'job_history', 'snapshots', 'skills', 'education', 'about',
       'updated', 'middle_name', 'num_connections', 'interests', 'volunteered',
       'certifications', 'languages', 'recommendations', 'publications',
       'patents', 'industry', 'titles', 'company_name_complete',
       'job_title_complete'],
      dtype='object')

In [9]:
dfe.shape

(2566, 29)

In [10]:
# REGEX DE BOOTCAMPS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'dev f\\.'
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', 
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', 
 'nivel[ -]?pro ', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'reprograma',
 'worldtech', 
 'bictia|bi.?tia', 
 'talento[ -]?digital']

# EXTRAER FORMACIÓN DESDE EL DATASET DE LOS NO-MISSING VALUES DE EDUCACIÓN

In [11]:
#CONVERTIR EN STRING
dfe["education"] =  dfe["education"].str.lower()

In [12]:
#CONVERTIR EN DICCIONARIOS LA EDUCACIÓN
L = []

for i in dfe["education"]:
    l = str(i).split("},") #convertir en string la formación del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for formacion in l: #para cada formación
        formacion = formacion.replace("[", "").replace("]", "") + "}" #separelas y conviertalas en diccionarios
        k.append(formacion) #añadalas a la lista de formación
    L.append(k) #añada la lista de formación para individuo

#VER UN EJEMPLO: Esta es la última formación del primer individuo desordenada en diccionario pero como un string   
L[0][0]

"{'education_org': 'university of oxford', 'degrees': 'university of oxford – oxford royale academy, english and culture, common european framework of reference: profiecient user', 'end': '2013', 'start': '2013'}"

In [13]:
K = [] #LISTA D ELISTAS EN DONDE SE ALMACENARAN LAS FORMACIONES A MANERA DE DICCIONARIO PARA CADA INDIVIDUO

for individuo in L: #para cada individuo

    educacion_ind = [] #creo una lista de sus educaciones

    for ins in individuo: # para cada cada formación dentro de las formaciones de cada individuos
        ins = ins.replace('"', "'")

        c = {}

        #CADA CAMPO SE EXTRAE POR MEDIO DE RegEx, COMUNES A TODOS LAS FORMACIONES STR DE LOS INDIVIDUOS EN EL DATASET
        #CADA CAMPO SE EXTRAE EN CASO DE QUE EXISTA DE LO CONTRARIO NO

        #EXTRAER EDUCACIÓN
        if ( len(re.findall("'education_org': '.*?'", ins)) ) > 0:
            c["education"] =  re.findall("'education_org': '.*?'", ins)[0].replace("'education_org': ", "").replace("'", "")

        #EXTRAER DEGREE
        if ( len(re.findall("'degrees': '.*?'", ins)) ) > 0:
            c["degrees"] =  re.findall("'degrees': '.*?'", ins)[0].replace("'degrees': ", "").replace("'", "")

        #EXTRAER SUMMARY
        if ( len(re.findall("'summary': '.*?'", ins)) ) > 0:
            c["summary"] =  re.findall("'summary': '.*?'", ins)[0].replace("'summary': ", "").replace("'", "")


        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general

#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
K[0][0]

{'education': 'university of oxford',
 'degrees': 'university of oxford – oxford royale academy, english and culture, common european framework of reference: profiecient user',
 'start': '2013',
 'end': '2013'}

In [14]:
#CREAR COLUMNA LIMPIA DE EDUCACIÓN
dfe["educacion"] = K

In [15]:
#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
dfe.educacion[0][2]

{'education': 'vrije universiteit amsterdam',
 'degrees': '(pre)-master, organization science',
 'summary': '(schakeljaar tussen hbo-opleiding en masterstudie)\\n afstudeeronderzoek bij het ministerie van sociale zaken en werkgelegenheid(szw)\\n onderzoek op welke manier(en) de invoering van ‘het nieuwe werken’ bij het ministerie van szw, invloed heeft (gehad) op de betrokkenheid van de werknemers bij het uitvoeren hun werkzaamheden.\\n activities and societies: lid van faculteitsvereniging eos',
 'start': '2011',
 'end': '2012'}

In [66]:
lista_boots = ['4Geeks',
 'Acámica',
 'Ada ITW',
 'Bedu Tech',
 'Bogotá Dev',
 'Bootcamp Institute',
 'Coderise International',
 'Desafío Latam',
 'Dev F.',
 'Digital House',
 'Generation Mexico',
 'Hack Academy',
 'HolaCode',
 'Holberton School',
 'iCreate Coding Academy',
 'IJALTI',
 'Iron Hack',
 'Jóvenes a Programar',
 'Keep Coding',
 'Kod1go',
 'Laboratoria',
 'Le Wagon',
 'Make it Real Camp',
 'MindHub ',
 'Muktek Academy',
 'Nivel Pro',
 'Noukod',
 'Plataforma 5',
 'Programa Valentina',
 'Pygmalion',
 'Reprograma',
 'World Tech Inc (a.k.a. World Tech Makers –WTM–)',
 'BICTIA',
 'Talento Digital']

In [70]:
# REGEX DE BOOTCAMPS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', 
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding|keep.?coding.io',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', 
 'nivel[ -]?pro ', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'pygmalion'
 'reprograma',
 'worldtech$',  #se cambió  para que solo termine con $
 'bictia|bi.?tia', 
 'talento[ -]?digital$'] #

 #VERDE: MÁS GRANDES DE LA, #AMARILLOS: MEDIANOS
 #EN PRINCIPIOS LOS VERDES SON LOS QUE DEBERIAN APARECER MÁS EN LOS PERFILES, EN PRINCIPIO.


 #BASE: LINKED-IN: + DIFICIL PARA HACER SCRAPING. EMPRESAS VENDEN DATOS (OBTUVIERON HACIENDO SCRAPING), PERFILES EGRESADOS DE BOOTCAMPS DE LISTA DE BOOTCAMPS EN LA. 3 RONDAS DE INTENTOS Y MANDAN MUESTRA PEQUEÑA. SUGIEREN MUCHOS DE LOS CAMBIOS.abs

 #NEGOCIACIÓN, PERFILES Y CUÁNTO CUESTA. MAX 8MIL. CADA FALSO POSITIVO ES UN PROBLEMA. 
 # LLEGABAN A MIL, PARECIA QUE FP NO ERAN PROBLEMA: MEJOR ESTRATEGIA PALABRAS GENERALES Y TRAEMOS 5MIL PERFILES Y DENTRO DE ESA DEBEN DE ESTAR TODOS ESTUDIANTES DE PERFILES PROFESIONALES DE BOOTCAMPS.
 # EGRESADOS EN LATIRNOAMERICA DEBERÍAN SER MUCHO MÁS DE 8MIL, EJ: DIGITAL HOUSE HABLAN DE 100MIL EGRESADOS

 #NO REPRESENTATIVIDAD: MUCHOS EGRESADOS POCA MUESTRA, NO TODOS TIENEN PERFIL DE LINKED-IN

 #PRIORIDAD

In [71]:
#LOOP PARA BUSCAR SI EL INDIVIDUO TIENE FORMACIÓN EN ALGÚN BOOTCAMP
c = []
for boot in range(len(boots)): #PARA CADA BOOTCAMP
    for i in range(len(dfe.educacion)): #PARA CADA INDIVIDUO
        for h in range(len(dfe.educacion[i])): #DENTRO LA EDUCACIÓN DE CADA INDIVIDUO
            if "education" in dfe.educacion[i][h]: #SI TIENE EL CAMPO EDUCACIÓN
                A = re.findall(boots[boot], dfe.educacion[i][h]["education"]) #REVISA SI ESTÁ EN UN BOOTCAMP
                if len(A) > 0 : #SI ESA REVISIÓN ES NO VACÍA
                    c.append((dfe.id[i], boots[boot], lista_boots[boot], dfe.educacion[i][h]['education']))
                    # c.append((dfe.id[i], h)) #AÑADE EL ID DE ESE INDIVIDUO Y EL LUGAR DONDE ESTÁ ESA EDUCACIÓN
                    print(f"{dfe['job_title'][i]}---{A}---{boots[boot]}--{dfe.educacion[i][h]['education']} \n \n")


            #LOOP SIN INDICE
            # for edu in dfe.educacion[i]:
            # if "education" in edu:
            #     # print(boot, edu["education"])
            #     A = re.findall(boot, edu["education"])
            #     if len(A) > 0:
            #         c.append(A)
            #         m.append(boot)
            #         print(f" {dfe['job_title'][i]} -- {A} -- {boot} -- {edu['education']} \n \n")
            #         # print(f"{A} -- {boot} -- {edu['education']}")

CEO---['acamica']---ac.mica--acamica 
 

Co-founder & CEO---['acamica']---ac.mica--acamica 
 

Web Full Stack Developer---['acamica']---ac.mica--acamica 
 

Ingeniero---['acamica']---ac.mica--acamica 
 

Senior Javascript Developer---['acamica']---ac.mica--codecademy/acamica 
 

Technical Support Engineer---['acamica']---ac.mica--acamica 
 

Lider de Ecommerce---['acamica']---ac.mica--acamica 
 

API - UI Automation---['acamica']---ac.mica--globant & acamica 
 

Coordinadora de Marketing Digital---['acamica']---ac.mica--acamica.com 
 

Student---['acamica']---ac.mica--acamica 
 

Student---['acamica']---ac.mica--acamica 
 

Solution Developer. Frontend Developer (Angular)---['acamica']---ac.mica--acamica 
 

Solution Developer. Frontend Developer (Angular)---['acamica']---ac.mica--acamica 
 

UX/UI Designer---['acamica']---ac.mica--acamica 
 

UX/UI Designer---['acamica']---ac.mica--acamica 
 

UX/UI Designer---['acamica']---ac.mica--acamica 
 

Sr UI/UX Designer---['acamica']---ac.mic

In [72]:
c = list(set(c))
educ_boots = pd.DataFrame.from_records
    c, columns=['id', 'regex_bootcamp', 'bootcamp', "educ_title"])

In [74]:
educ_boots = pd.merge(educ_boots, dfe, "left", "id")
educ_boots

,id,regex_bootcamp,bootcamp,educ_title,scraping_services_id,linkedin_id,linkedin_profile_url,last_name,first_name,v_extension,...,certifications,languages,recommendations,publications,patents,industry,titles,company_name_complete,job_title_complete,educacion
0,47709881,plataforma[ -]?5,Plataforma 5,plataforma 5,47709881,gabrielaavilamarketingdigital,https://www.linkedin.com/in/gabrielaavilamarke...,Avila,Gabriela,"{'gender': 'female', 'industry_from_job': None...",...,NaN,"['Ingles', 'Español', 'Portugues', 'Italiano']","['Gabriela es una diseñadora muy completa, log...",NaN,NaN,NaN,NaN,NaN,NaN,"[{'education': 'universidad de buenos aires', ..."
1,13408816,digital[ -]?house,Digital House,digital house,13408816,maría-fernanda-fabris-4a211053,https://www.linkedin.com/in/maría-fernanda-fab...,Fabris,María,"{'gender': 'female', 'industry_from_job': 'Mar...",...,"[{'end': 'Present', 'title': 'Language Certifi...","['Chino', 'Ingles']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'education': 'digital house', 'degrees': 'di..."
2,68182368,keep.?coding|keep.?coding.io,Keep Coding,keepcoding,68182368,dtorguet,https://www.linkedin.com/in/dtorguet,Torguet,David,"{'gender': 'male', 'industry_from_job': 'Infor...",...,"[{'certificate_authority': 'Udemy', 'end': 'Pr...","['English', 'French', 'Spanish', 'Catalan']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'education': 'ioc', 'degrees': 'dai, desarro..."
3,49105504,ac.mica,Acámica,codecademy/acamica,49105504,igmartin1,https://www.linkedin.com/in/igmartin1,Martin,Ignacio,"{'gender': 'male', 'industry_from_job': 'Compu...",...,NaN,"['Spanish', 'English']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{'education': 'universidad nacional de mar de...
4,64842254,digital[ -]?house,Digital House,digital house,64842254,matiascutufia,https://www.linkedin.com/in/matiascutufia,Cutufia,Matías,"{'gender': 'male', 'industry_from_job': False,...",...,[{'certificate_authority': 'Coursera Verified ...,"['Español', 'Inglés', 'Portugués']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{'education': 'universidad tecnológica nacion...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,32583055,ac.mica,Acámica,acamica,32583055,lautaro-piacquadio-748790106,https://www.linkedin.com/in/lautaro-piacquadio...,Piacquadio,Lautaro,"{'gender': 'male', 'industry_from_job': 'Compu...",...,NaN,"['Español', 'Inglés']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{'education': 'universidad argentina de la em...
96,77852405,ac.mica,Acámica,acamica,77852405,gustavodanielromerosantos,https://www.linkedin.com/in/gustavodanielromer...,Santos,Gustavo,"{'gender': 'male', 'industry_from_job': 'Manag...",...,NaN,"['Inglés', 'Portugués', 'Spanish']",['He conocido a Gustavo fuera del ambiente lab...,[{'authors': [{'full_name': 'Gustavo Daniel Ro...,NaN,NaN,NaN,NaN,NaN,[{'education': 'apep asociación civil programa...
97,48959188,digital[ -]?house,Digital House,digital house,48959188,pedro-carafi-775177b1,https://www.linkedin.com/in/pedro-carafi-775177b1,Carafi,Pedro,"{'gender': 'male', 'industry_from_job': False,...",...,NaN,"['Español', 'Inglés']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'education': 'digital house', 'degrees': 'ma..."
98,50609046,keep.?coding|keep.?coding.io,Keep Coding,keepcoding,50609046,josé-sánchez-rodríguez-b8a509105,https://www.linkedin.com/in/josé-sánchez-rodrí...,Rodríguez,José,"{'gender': 'male', 'industry_from_job': None, ...",...,NaN,"['Español', 'Inglés']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'education': 'ies leonardo da vinci', 'degre..."


In [ ]:
df_educ 